In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [92]:
# read the data from drive
df = pd.read_csv('/content/drive/My Drive/XAI/CF/healthcare-dataset-stroke-data.csv')

In [93]:
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [94]:
df.drop('id', axis=1, inplace=True)
df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
5105,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [95]:
df.isnull().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [96]:
'''
BMI has 201 records of null value, as we can see that total number of records has 5110 number of rows, we can drop these null value.
But, if is better to check the distribution of null value with response varaible, 
Lets first check null values distribution
'''

print('Distribution of response varaible:\n', df['stroke'].value_counts())
print('Distribution of response value of null value:\n', df.query('bmi=="NaN"')['stroke'].value_counts())

'''
We can see that the data is highly inbalance, the dataset contain only 249 records for stoke patients where as 40 records has null vlaue in bmi
if we drop the null values then the stroke patient records reduce by 40, will contain only 209 records

Therefore it is better to fill these null values in an appropriate way
'''

Distribution of response varaible:
 0    4861
1     249
Name: stroke, dtype: int64
Distribution of response value of null value:
 0    161
1     40
Name: stroke, dtype: int64


'\nWe can see that the data is highly inbalance, the dataset contain only 249 records for stoke patients where as 40 records has null vlaue in bmi\nif we drop the null values then the stroke patient records reduce by 40, will contain only 209 records\n\nTherefore it is better to fill these null values in an appropriate way\n'

In [97]:
gen_age_bmi=df.query('bmi!="NaN"')[['bmi','gender', 'age']].groupby(['gender', 'age']).mean()
gen_age_bmi

bmi
gender age             
Female 0.08   14.100000
       0.32   17.266667
       0.40   17.400000
       0.48   16.100000
       0.56   18.300000
...                 ...
Male   79.00  27.681818
       80.00  29.210714
       81.00  27.677273
       82.00  27.943478
Other  26.00  22.400000

[205 rows x 1 columns]

In [98]:
def fill_bmi(row):
  g = row['gender']
  a = row['age']
  
  if a in gen_age_bmi.loc[g].index:
    bmi = np.mean(gen_age_bmi.loc[g, a].values)
  else:
    idx = np.array(gen_age_bmi.loc[g].index)
    l = idx[idx<a][-1] if idx[idx<a][-1] else 0
    u = idx[idx>a][0] if idx[idx>a][0] else 0

    if l!=0 and u!=0: bmi=(np.mean(gen_age_bmi.loc[g,l])+np.mean(gen_age_bmi.loc[g, u]))/2
    elif l == 0: bmi = np.mean(gen_age_bmi.loc[g, u])
    else: bmi = np.mean(gen_age_bmi.loc[g,l])

  row['bmi'] = bmi 
  return row

In [99]:
# bmi_null = df.query('bmi=="NaN"')
bmi_not_null = df.query('bmi!="NaN"')
bmi_null = df.query('bmi=="NaN"').apply(lambda x: fill_bmi(x), axis=1)
df = pd.concat([bmi_not_null, bmi_null], axis=0)

In [100]:
df.isnull().sum(), df.shape

(gender               0
 age                  0
 hypertension         0
 heart_disease        0
 ever_married         0
 work_type            0
 Residence_type       0
 avg_glucose_level    0
 bmi                  0
 smoking_status       0
 stroke               0
 dtype: int64, (5110, 11))

In [101]:
df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1
5,Male,81.0,0,0,Yes,Private,Urban,186.21,29.000000,formerly smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
5039,Male,41.0,0,0,No,Private,Rural,70.15,32.716000,formerly smoked,0
5048,Male,40.0,0,0,Yes,Private,Urban,191.15,28.313636,smokes,0
5093,Female,45.0,1,0,Yes,Govt_job,Rural,95.02,31.459322,smokes,0
5099,Male,40.0,0,0,Yes,Private,Rural,83.94,28.313636,smokes,0


In [102]:
df1 = df.query('stroke==1')
df0 = df.query('stroke==0')
df0 = df0.sample(frac=0.2, replace=True, random_state=1)
df_reduce = pd.concat([df0, df1], axis=0)
df_reduce.shape

(1221, 11)

In [104]:
df_reduce['stroke'].value_counts()

0    972
1    249
Name: stroke, dtype: int64

In [105]:
# Split data into train and test
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier

# Seperate features and target
target = df_reduce["stroke"]
datasetX = df_reduce.drop("stroke", axis=1)

# Split into train and test
x_train, x_test, y_train, y_test = train_test_split(datasetX, target, test_size = 0.3, random_state=42, stratify=target)

# Seperate numerical and categorial features
numerical=['age', 'avg_glucose_level', 'bmi']
categorical = x_train.columns.difference(numerical) 

# Transform categorial features into one hot endoced form
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
transformations = ColumnTransformer(transformers=[('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.

pipe_rfc = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])

pipe_lr = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', LogisticRegression(random_state=42))])

pipe_svc = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', SVC())])

pipe_knn = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', KNeighborsClassifier())])

pipe_dtc = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', DecisionTreeClassifier())])                     

pipe_gbc = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', GradientBoostingClassifier())])

pipe_etc = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', ExtraTreesClassifier())])

pipelines = [pipe_rfc, pipe_lr, pipe_svc, pipe_knn, pipe_dtc, pipe_gbc, pipe_etc]

In [106]:
pile_model = {0:'Random Forest Classifier', 1:'Logistic Regression', 2:'Support Vector Classifier', 3:'KNeighborsClassifier',
              4:'Decision Tree Classifier', 5:'Gradient Boosting Classifier', 6:'Extra Trees Classifier'}

In [107]:
# Fit the pipelines
for pipe in pipelines:
	pipe.fit(x_train, y_train)

In [108]:
# Compare accuracies
for idx, val in enumerate(pipelines):
	print('######', pile_model[idx], '######\n', val.score(x_test, y_test),'\n')

###### Random Forest Classifier ######
 0.779291553133515 

###### Logistic Regression ######
 0.8010899182561307 

###### Support Vector Classifier ######
 0.8010899182561307 

###### KNeighborsClassifier ######
 0.8065395095367848 

###### Decision Tree Classifier ######
 0.7874659400544959 

###### Gradient Boosting Classifier ######
 0.784741144414169 

###### Extra Trees Classifier ######
 0.7929155313351499 



In [109]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [110]:
final = []
for idx, val in enumerate(pipelines):
  result = {}
  ypredic = val.predict(x_test)
  result['model'] = pile_model[idx]
  result['accuracy'] = accuracy_score(ypredic, y_test)
  result['precision'] = precision_score(ypredic, y_test)
  result['recall'] = recall_score(ypredic, y_test)
  result['f1-score'] = f1_score(ypredic, y_test)
  final.append(result)
result_df = pd.DataFrame(final)
result_df

,model,accuracy,precision,recall,f1-score
0,Random Forest Classifier,0.779292,0.120000,0.375000,0.181818
1,Logistic Regression,0.801090,0.106667,0.571429,0.179775
2,Support Vector Classifier,0.801090,0.066667,0.625000,0.120482
3,KNeighborsClassifier,0.806540,0.173333,0.590909,0.268041
4,Decision Tree Classifier,0.787466,0.120000,0.428571,0.187500
5,Gradient Boosting Classifier,0.784741,0.106667,0.400000,0.168421
6,Extra Trees Classifier,0.792916,0.106667,0.470588,0.173913


In [111]:
for idx, val in enumerate(pipelines):
  print('######', pile_model[idx], '######')
  print(classification_report(val.predict(x_test), y_test), '\n')

###### Random Forest Classifier ######
              precision    recall  f1-score   support

           0       0.95      0.81      0.87       343
           1       0.12      0.38      0.18        24

    accuracy                           0.78       367
   macro avg       0.53      0.59      0.53       367
weighted avg       0.89      0.78      0.83       367
 

###### Logistic Regression ######
              precision    recall  f1-score   support

           0       0.98      0.81      0.89       353
           1       0.11      0.57      0.18        14

    accuracy                           0.80       367
   macro avg       0.54      0.69      0.53       367
weighted avg       0.95      0.80      0.86       367
 

###### Support Vector Classifier ######
              precision    recall  f1-score   support

           0       0.99      0.81      0.89       359
           1       0.07      0.62      0.12         8

    accuracy                           0.80       367
   macro av

In [136]:
# Find the best modle
np.argmax(result_df['accuracy'])

ValueError: ignored

In [112]:
s0 = 0
s1 = 0
for val in y_test:
  if val==0:
    s0 +=1
  else:
    s1 +=1
print('S0: ', s0, 'S1: ', s1)

S0:  292 S1:  75


In [116]:
# best model
model = pilelines[3]
model

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('cat',
                                                  Pipeline(memory=None,
                                                           steps=[('onehot',
                                                                   OneHotEncoder(categories='auto',
                                                                                 drop=None,
                                                                                 dtype=<class 'numpy.float64'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=True))],
                                                

# CounterFactual explanation

In [117]:
!pip install dice-ml

     |████████████████████████████████| 225kB 5.5MB/s 


In [118]:
import dice_ml
from dice_ml.utils import helpers

In [121]:
d = dice_ml.Data(dataframe=df_reduce, continuous_features=['age', 'avg_glucose_level', 'bmi'], outcome_name='stroke')

In [122]:
model = model.fit(x_train, y_train)

In [123]:
# Using sklearn backend
m = dice_ml.Model(model=model, backend="sklearn")
# Using method=random for generating CFs
exp = dice_ml.Dice(d, m, method="random")

In [124]:
e1 = exp.generate_counterfactuals(x_train[300:301], total_CFs=2, desired_class="opposite")
e1.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Female,53.0,0,0,Yes,Private,Urban,83.41,29.9,never smoked,0



Diverse Counterfactual set (new outcome: 1.0)


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,-,-,-,1,-,-,Rural,-,-,-,1
1,-,-,-,0,-,-,Rural,-,-,-,1


In [125]:
# Lets find the index of patients who has stroke
idx = np.where(y_train==1)

In [126]:
idx

(array([  0,   2,  15,  20,  21,  27,  31,  33,  38,  44,  45,  47,  48,
         51,  58,  62,  65,  72,  79,  88,  93, 101, 107, 130, 136, 140,
        145, 147, 151, 162, 171, 178, 184, 197, 198, 204, 208, 216, 219,
        220, 225, 227, 232, 234, 253, 255, 257, 260, 263, 271, 273, 281,
        282, 284, 287, 289, 290, 295, 296, 299, 304, 314, 319, 320, 322,
        326, 328, 331, 339, 340, 343, 356, 360, 362, 368, 370, 377, 379,
        383, 384, 402, 404, 407, 413, 416, 421, 427, 432, 435, 436, 437,
        445, 447, 452, 453, 457, 462, 465, 467, 470, 471, 480, 485, 486,
        501, 509, 521, 526, 531, 533, 536, 544, 545, 546, 548, 549, 553,
        555, 561, 569, 572, 576, 578, 595, 600, 604, 608, 611, 621, 624,
        626, 631, 646, 657, 662, 667, 668, 673, 677, 681, 688, 693, 699,
        700, 702, 708, 709, 711, 714, 731, 734, 753, 754, 756, 760, 765,
        768, 775, 777, 779, 782, 791, 794, 806, 808, 815, 818, 820, 826,
        828, 833, 834, 839, 849]),)

In [128]:
e1 = exp.generate_counterfactuals(x_train[0:1], total_CFs=4, desired_class="opposite")
e1.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 1)


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,80.0,1,1,Yes,Private,Urban,175.29,31.5,formerly smoked,1



Diverse Counterfactual set (new outcome: 0.0)


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,-,-,-,-,No,-,-,-,29.1,-,0
1,-,-,-,-,No,-,-,238.32,-,-,0
2,-,-,-,-,-,Never_worked,-,210.11,-,-,0
3,-,-,-,-,No,Govt_job,-,-,-,-,0


In [129]:
e1.visualize_as_dataframe(show_only_changes=False)

Query instance (original outcome : 1)


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,80.0,1,1,Yes,Private,Urban,175.29,31.5,formerly smoked,1



Diverse Counterfactual set (new outcome: 0.0)


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,80,1,1,No,Private,Urban,175.29,29.1,formerly smoked,0
1,Male,80,1,1,No,Private,Urban,238.32,31.5,formerly smoked,0
2,Male,80,1,1,Yes,Never_worked,Urban,210.11,31.5,formerly smoked,0
3,Male,80,1,1,No,Govt_job,Urban,175.29,31.5,formerly smoked,0


# Changing only age and hypertension

In [135]:
e2 = exp.generate_counterfactuals(x_train[0:1], 
                                  total_CFs=4, 
                                  desired_class="opposite",
                                  features_to_vary=["hypertension", "heart_disease", "age"]
                                  )
e2.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 1)


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,80.0,1,1,Yes,Private,Urban,175.29,31.5,formerly smoked,1



Diverse Counterfactual set (new outcome: 0.0)


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,-,-,0,1,-,-,-,-,-,-,0
1,-,-,1,0,-,-,-,-,-,-,0
2,-,-,1,1,-,-,-,-,-,-,0
3,-,-,0,0,-,-,-,-,-,-,0


# # Restricting age to be between [70, 80].

In [131]:
e3 = exp.generate_counterfactuals(x_train[0:1], 
                                  total_CFs=2, 
                                  desired_class="opposite",
                                  permitted_range={'age':[70,80]})
e3.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 1)


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,80.0,1,1,Yes,Private,Urban,175.29,31.5,formerly smoked,1



Diverse Counterfactual set (new outcome: 0.0)


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,-,-,-,-,No,-,-,-,-,smokes,0
1,-,-,-,-,No,-,-,-,14.2,-,0


# Local feature importance scores

In [132]:
query_instance=x_train[0:1]
imp = exp.local_feature_importance(query_instance, total_CFs=10)
print(imp.local_importance)

[{'smoking_status': 0.6, 'ever_married': 0.3, 'bmi': 0.3, 'hypertension': 0.2, 'work_type': 0.2, 'age': 0.2, 'gender': 0.1, 'heart_disease': 0.1, 'Residence_type': 0.0, 'avg_glucose_level': 0.0}]


# Global feature importance scores

In [134]:
query_instances=x_train[0:25]
imp = exp.global_feature_importance(query_instances)
print(imp.summary_importance)

{'heart_disease': 0.636, 'work_type': 0.544, 'smoking_status': 0.444, 'hypertension': 0.34, 'Residence_type': 0.336, 'bmi': 0.276, 'age': 0.272, 'ever_married': 0.268, 'avg_glucose_level': 0.232, 'gender': 0.132}
